# RDD

an RDD is the fundamental data structure of Apache Spark. It's a fault-tolerant, distributed collection of elements that can be operated on in parallel.

**Key Characteristics:**

- Immutable
- Lazy evaluation
- Fault tolerant (via lineage info)
- Partitioned across cluster nodes
- Can be cached in memory

### SparkContext and SparkConf


SparkContext is the entry point for Spark functionality.

#### `SparkConf`

- Configuration for Spark application

**Common settings:**

- setMaster("local[*]") – Use local mode with all cores
- setAppName("RDDExample") – Application name

### transformations

Transformations create a new RDD from an existing one. They are lazy – not executed until an action is triggered.

| Transformation  | Description                                          |
| --------------- | ---------------------------------------------------- |
| `map(func)`     | Returns a new RDD by applying `func` to each element |
| `filter(func)`  | Filters elements for which `func` returns true       |
| `flatMap(func)` | Like map but flattens the result                     |
| `distinct()`    | Removes duplicates                                   |
| `union(rdd)`    | Combines two RDDs                                    |
| `groupByKey()`  | Groups values with same key                          |
| `reduceByKey()` | Aggregates values with same key using a function     |
| `sortBy(func)`  | Sorts RDD by computed key                            |


### actions

Actions trigger computation and return results or write data.

| Action             | Description                            |
| ------------------ | -------------------------------------- |
| `collect()`        | Returns all elements to driver         |
| `count()`          | Returns number of elements             |
| `first()`          | Returns first element                  |
| `take(n)`          | Returns first `n` elements             |
| `reduce(func)`     | Reduces elements using binary operator |
| `saveAsTextFile()` | Writes RDD to text files               |



reference - [spark rdd docs](https://spark.apache.org/docs/latest/rdd-programming-guide.html)


In [60]:
! pip install pyspark

# pyspark 4.0.0 - jdk 17

Defaulting to user installation because normal site-packages is not writeable


In [61]:
! pip install pyspark=3.3.0 

# pyspark 3.3.* - jdk 11

Defaulting to user installation because normal site-packages is not writeable
ERROR: Invalid requirement: 'pyspark=3.3.0'
Hint: = is not a valid operator. Did you mean == ?


In [62]:
# SparkContext and SparkConf

from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("salesDemo").setMaster("local[*]")

sc = SparkContext(conf=conf)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=salesDemo, master=local[*]) created by __init__ at /tmp/ipykernel_6478/4167189320.py:7 

In [ ]:
sc.defaultParallelism

1

In [ ]:
# Step 1: Load the file into RDD

sales_raw = sc.textFile("file:////workspace/TRNG-2224-data-engineering/week1/datasets/sales.txt")

sales_raw.collect()



['1001,Furniture,69',
 '1002,Clothing,78',
 '1003,Electronics,821',
 '1004,Furniture,357',
 '1005,Furniture,429',
 '1006,Furniture,540',
 '1007,Books,724',
 '1008,Electronics,770',
 '1009,Furniture,503',
 '1010,Clothing,331',
 '1011,Clothing,227',
 '1012,Toys,400',
 '1013,Clothing,459',
 '1014,Electronics,987',
 '1015,Books,627',
 '1016,Electronics,481',
 '1017,Books,314',
 '1018,Furniture,803',
 '1019,Electronics,453',
 '1020,Toys,800',
 '1021,Clothing,894',
 '1022,Furniture,176',
 '1023,Electronics,352',
 '1024,Electronics,930',
 '1025,Toys,584',
 '1026,Electronics,248',
 '1027,Toys,243',
 '1028,Toys,861',
 '1029,Books,229',
 '1030,Toys,618',
 '1031,Electronics,246',
 '1032,Toys,829',
 '1033,Clothing,329',
 '1034,Clothing,427',
 '1035,Toys,101',
 '1036,Clothing,330',
 '1037,Furniture,567',
 '1038,Toys,542',
 '1039,Electronics,941',
 '1040,Clothing,87',
 '1041,Furniture,514',
 '1042,Books,668',
 '1043,Electronics,545',
 '1044,Toys,558',
 '1045,Electronics,616',
 '1046,Books,412',
 '10

In [ ]:
# Step 2: Convert each line into a tuple (ProductID, Category, Amount)

records = sales_raw.map(lambda x: x.split(",")).map(lambda x: (int(x[0]), x[1], int(x[2])))




In [ ]:
# Step 3: Create a (Category, Amount) RDD

category_sales = records.map(lambda x: (x[1], x[2]) )


In [ ]:
# Step 4: Total sales by category

total_sales_by_category = category_sales.reduceByKey(lambda x,y: x+y)


In [ ]:
# Step 5: Average sale per category

average_sales_by_category = category_sales.mapValues(lambda x: (x, 1)).reduceByKey(lambda x,y: (x[0] + y[0], x[1]+ y[1])).mapValues(lambda x: x[0]/x[1])

average_sales_by_category.take(4)


[('Furniture', 538.4090909090909),
 ('Clothing', 443.0),
 ('Electronics', 547.4166666666666),
 ('Books', 407.3636363636364)]

In [ ]:
# Step 6: Highest transaction

max_tran = records.max(key=lambda x : x[2])

max_tran


(1014, 'Electronics', 987)

In [ ]:
# highest selling category

highest_selling_category = total_sales_by_category.max(key=lambda x: x[1] )

highest_selling_category

('Electronics', 13138)

In [ ]:
# Step 7: Categories with sales above 5000

high_selling_cat_5k = total_sales_by_category.filter(lambda x: x[1] > 5000)

high_selling_cat_5k.collect()




[('Furniture', 11845),
 ('Clothing', 9303),
 ('Electronics', 13138),
 ('Toys', 11794)]

In [ ]:
# print final results

print("Total sales by cat")
print(total_sales_by_category.collect())

print("Avg sales per cat")

print(average_sales_by_category.collect())



Total sales by cat
[('Furniture', 11845), ('Clothing', 9303), ('Electronics', 13138), ('Books', 4481), ('Toys', 11794)]
Avg sales per cat
[('Furniture', 538.4090909090909), ('Clothing', 443.0), ('Electronics', 547.4166666666666), ('Books', 407.3636363636364), ('Toys', 536.0909090909091)]


**Assignment:**

#1. find all product IDs where the amount is greater than 900.

In [ ]:

high_end_products = records.filter(lambda x: x[2] > 900)
high_end_products_ids = high_end_products.map(lambda x: x[0])
print(high_end_products_ids.collect())

[1014, 1024, 1039]


2. Find all transactions that belong to the “Furniture” category.


In [ ]:
furniture_transactions = records.filter(lambda x: x[1] == 'Furniture' )
print(furniture_transactions.collect())

[(1001, 'Furniture', 69), (1004, 'Furniture', 357), (1005, 'Furniture', 429), (1006, 'Furniture', 540), (1009, 'Furniture', 503), (1018, 'Furniture', 803), (1022, 'Furniture', 176), (1037, 'Furniture', 567), (1041, 'Furniture', 514), (1047, 'Furniture', 527), (1048, 'Furniture', 364), (1051, 'Furniture', 168), (1056, 'Furniture', 860), (1057, 'Furniture', 727), (1060, 'Furniture', 734), (1061, 'Furniture', 586), (1064, 'Furniture', 653), (1066, 'Furniture', 425), (1068, 'Furniture', 633), (1070, 'Furniture', 879), (1082, 'Furniture', 645), (1084, 'Furniture', 686)]


3. Count how many transactions belong to the “Electronics” category.


In [ ]:
electronics_transactions = records.filter(lambda x: x[1] == 'Electronics' )
electronics_transactions.collect()
count_electronics_transactions = electronics_transactions.count()
print(count_electronics_transactions)

24


4. Find average amount for each category.

In [ ]:
average_sales_by_category.collect()

[('Furniture', 538.4090909090909),
 ('Clothing', 443.0),
 ('Electronics', 547.4166666666666),
 ('Books', 407.3636363636364),
 ('Toys', 536.0909090909091)]


5. Find the highest amount and the corresponding product ID.


In [ ]:
max_tran = records.max(key=lambda x : x[2])
print(f"Highest amount {max_tran[2]} for product id {max_tran[0]}")

Highest amount 987 for product id 1014


6. Find the total number of unique categories.


In [ ]:
print(f"Unique category count: {average_sales_by_category.count()}")

Unique category count: 5


7. For each category, find the product ID with the highest sale.


8. Count how many products were sold for less than 300.


In [63]:
cheap_products = records.filter(lambda x: x[2] < 300)
print(cheap_products.count())

22


9. Sort the transactions by amount in descending order.


In [66]:
sorted = records.sortBy(lambda x: x[2], False)
print(sorted.collect())


[(1014, 'Electronics', 987), (1039, 'Electronics', 941), (1024, 'Electronics', 930), (1021, 'Clothing', 894), (1069, 'Clothing', 883), (1081, 'Electronics', 881), (1070, 'Furniture', 879), (1028, 'Toys', 861), (1056, 'Furniture', 860), (1078, 'Electronics', 845), (1058, 'Toys', 836), (1032, 'Toys', 829), (1003, 'Electronics', 821), (1018, 'Furniture', 803), (1020, 'Toys', 800), (1073, 'Toys', 800), (1076, 'Clothing', 797), (1092, 'Electronics', 778), (1098, 'Electronics', 776), (1008, 'Electronics', 770), (1060, 'Furniture', 734), (1057, 'Furniture', 727), (1007, 'Books', 724), (1097, 'Clothing', 716), (1055, 'Toys', 693), (1084, 'Furniture', 686), (1088, 'Toys', 683), (1072, 'Toys', 678), (1042, 'Books', 668), (1064, 'Furniture', 653), (1065, 'Electronics', 651), (1091, 'Clothing', 647), (1082, 'Furniture', 645), (1068, 'Furniture', 633), (1015, 'Books', 627), (1030, 'Toys', 618), (1045, 'Electronics', 616), (1094, 'Toys', 587), (1061, 'Furniture', 586), (1025, 'Toys', 584), (1080, 'C

### Shared Variables

When you pass a function (like in map or reduce) to Spark, that function runs on different machines in the cluster — not on your local driver.

**By default:**

- Spark makes separate copies of any variable you use inside those functions.
- So, if a task changes a variable on the executor, that change will not reflect back in your driver program.

This is done to keep things fast and distributed — but it also means you can not just update normal variables across tasks.

**challenges:**

You want to count how many rows have Amount > 500 using this code:

```py
count = 0

def increment_count(x):
    count += 1 

records.filter(lambda x: int(x[2]) > 500).foreach(increment_count)
print(count)

```

This will not work because each machine updates its own copy of count not the original one in the driver.

To solve such issues, spark provides two types of shared variables:

1. **Broadcast Variables:** a read-only variable that can be cached on each machine (executor). Used to efficiently share large data (like lookup tables) with all tasks without copying it multiple times.
2. **Accumulators:** variables used to safely implement counters or sums across mulitple worker nodes. You can only add to them (not read or subtract inside tasks). The final value is only accessible on the driver after an action is executed.




In [67]:
# Load the dataset


region_sales = sc.textFile("file:///workspace/TRNG-2224-data-engineering/week1/datasets/region_sales.txt")




In [68]:
# Use a broadcast variable for region-level config

records_rs = region_sales.map(lambda x: x.split(",")).map(lambda x: (int(x[0]), x[1], x[2], int(x[3])))

regions = {"North": "N", "South": "S", "East": "E", "West": "W"}

region_broadcast = sc.broadcast(regions)


In [69]:
# Use an accumulator to count high-value transactions

high_value_count = sc.accumulator(0)




In [70]:
def enrich_region_and_count(record):
    pid, region, category, amount = record
    if amount > 800:
        high_value_count.add(1)
    return (pid, region_broadcast.value[region], category, amount)

enriched_records = records_rs.map(enrich_region_and_count)

In [71]:
# Save filtered RDD to a new file

filtered = enriched_records.filter(lambda x: x[3] >800)

filtered.map(lambda x: ",".join(map(str,x))).coalesce(1).saveAsTextFile("filtered_high_value_sales")




Py4JJavaError: An error occurred while calling o675.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/workspace/TRNG-2224-data-engineering/week1/filtered_high_value_sales already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:303)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:73)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1094)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.scala:18)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:417)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1092)
	at 

In [72]:
high_value_count.value

0

### Assignment

**tasks:**

1.  Load the Dataset
2. Broadcast Variable: Category Discounts

```py
{"Electronics": 0.10, "Furniture": 0.15, "Clothing": 0.05, "Books": 0.20}
```

3. calculate and return the rdd:

```py
(ProductID, Region, Category, OriginalAmount, DiscountApplied)
```
4. Accumulator: Count Transactions Below ₹300
5. Filter and Save Results

In [79]:
ds_raw = sc.textFile("file:////workspace/TRNG-2224-data-engineering/week1/datasets/region_sales.txt")
ds_raw.collect()
region_sales_records = ds_raw.map(lambda x: x.split(",")).map(lambda x: (int(x[0]), x[1], x[2], int(x[3])))

cat_discounts = {"Electronics": 0.10, "Furniture": 0.15, "Clothing": 0.05, "Books": 0.20}
cat_discounts_broadcast = sc.broadcast(cat_discounts)
low_trans_count = sc.accumulator(0)

def apply_discount(record):
    pid, region, category, amount = record
    discount = amount * cat_discounts_broadcast.value[category]
    new_record = (pid, region, category, amount, amount - discount)
    if (amount - discount) < 300:
        low_trans_count.add(1)
    return new_record

discounted_records = region_sales_records.map(apply_discount)
filtered = discounted_records.filter(lambda x: x[4] < 300)
print(low_trans_count.value)
filtered.map(lambda x: ",".join(map(str,x))).coalesce(1).saveAsTextFile("filtered_low_sale_discounts")

0
